In [2]:
import sys

import polars as pl
import torch
'''
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
https://developer.nvidia.com/cuda-12-8-0-download-archive
'''

MAC_DIR = '/Users/igwanhyeong/PycharmProjects/data_research/raw_data/'
WINDOW_DIR = 'C:/Users/USER/PycharmProjects/research/raw_data/'

if sys.platform == 'win32':
    DIR = WINDOW_DIR
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.version.cuda)
    print(torch.__version__)
    print(torch.cuda.get_device_name(0))
    print(torch.__version__)
else:
    DIR = MAC_DIR
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

save_dir = DIR + 'fit/20251109_LTB'


ModuleNotFoundError: No module named 'modeling_module'

In [6]:
pl.read_parquet(DIR + 'target_dyn_demand.parquet').select('oper_part_no').unique()

oper_part_no
str
"""TG15-0227A"""
"""C7274-81332"""
"""T5238-21115"""
"""83952048"""
"""04612-00750"""
…
"""T4930-46331"""
"""58600-6144-2"""
"""DY94-O00"""


In [7]:

save_dir = DIR + 'fit/20251115_LTB'

In [8]:
target_dyn_demand_monthly = pl.read_parquet(DIR + 'target_dyn_demand_monthly.parquet').sort(['oper_part_no', 'demand_dt'])
target_dyn_demand_monthly = (target_dyn_demand_monthly
                                .group_by('oper_part_no', maintain_order = True)
                                .map_groups(lambda g: g.with_columns(pl.arange(1, len(g) + 1).alias('sequence')))
                            )


filtered_target = (target_dyn_demand_monthly
                    .group_by('oper_part_no')
                    .agg(pl.col('sequence').max().alias('sequence_max'))
                    .filter(pl.col('sequence_max') > 43)
                    .select('oper_part_no')
                   ) # seq Q75

target_dyn_demand_monthly = (target_dyn_demand_monthly
                                .join(filtered_target, on = 'oper_part_no', how = 'right')
                                .select(['oper_part_no', 'demand_dt', 'sequence', 'demand_qty'])
                             )
target_dyn_demand_monthly


oper_part_no,demand_dt,sequence,demand_qty
str,i64,i64,f64
"""01023-50612""",201801,1,10.0
"""01023-50612""",201804,2,2.0
"""01023-50612""",201805,3,20.0
"""01023-50612""",201808,4,50.0
"""01023-50612""",201809,5,101.0
…,…,…,…
"""Y173-A0020""",202607,83,665.0
"""Y173-A0020""",202609,84,1205.0
"""Y173-A0020""",202612,85,3150.0


In [9]:
from modeling_module.data_loader.MultiPartExoDataModule import MultiPartExoDataModule

plan_yyyymm = 201801
lookback = 12
horizon = 3


data_module = MultiPartExoDataModule(
    target_dyn_demand_monthly,
    lookback = lookback,
    horizon = horizon,
    batch_size = 128,
    val_ratio = 0.2,
    past_exo_cont_cols=('sequence',),
    is_running = False
)

train_loader = data_module.get_train_loader()
val_loader = data_module.get_val_loader()

[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat col: site_id -> skip
[TrainingDataset] missing past_exo_cat c

In [10]:
from modeling_module.training.model_trainers.total_train import run_total_train_monthly

model_dict = run_total_train_monthly(
    train_loader,
    val_loader,
    lookback = lookback,
    horizon = horizon,
    save_dir = save_dir
)

PatchMixer Base (Monthly)
[EXO-setup] inferred E=2, model.exo_dim=2, has_head=True

[train_patchmixer] ===== Stage 1/2 =====
  - spike: OFF
  - epochs: 50 | lr=0.0003 | horizon_decay=False
[train_patchmixer] Effective TrainingConfig:
{
  "device": "mps",
  "lookback": 12,
  "horizon": 3,
  "epochs": 50,
  "lr": 0.0003,
  "weight_decay": 0.001,
  "t_max": 40,
  "patience": 100,
  "max_grad_norm": 30.0,
  "amp_device": "cuda",
  "loss_mode": "point",
  "point_loss": "huber",
  "huber_delta": 0.8,
  "q_star": 0.5,
  "use_cost_q_star": false,
  "Cu": 1.0,
  "Co": 1.0,
  "quantiles": [
    0.1,
    0.5,
    0.9
  ],
  "use_intermittent": true,
  "alpha_zero": 3.0,
  "alpha_pos": 1.0,
  "gamma_run": 0.3,
  "cap": null,
  "use_horizon_decay": false,
  "tau_h": 0.85,
  "val_use_weights": false,
  "spike_loss": {
    "enabled": false,
    "strategy": "mix",
    "huber_delta": 0.6,
    "asym_up_weight": 1.0,
    "asym_down_weight": 8.0,
    "mad_k": 1.5,
    "w_spike": 32.0,
    "w_norm": 1.0,
 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


[EXO-batch] exo normalized to shape=(128, 3, 2) (expect [B,H,E])
[DEBUG] grad mean abs = 13.922868
[DEBUG] grad mean abs = 20.984692
[DEBUG] grad mean abs = 8.282863
[DEBUG] grad mean abs = 10.146101
[DEBUG] grad mean abs = 10.260278
[DEBUG] grad mean abs = 27.912746
[DEBUG] grad mean abs = 19.638469
[DEBUG] grad mean abs = 11.792342
[DEBUG] grad mean abs = 4.630276
[DEBUG] grad mean abs = 52.317214
[DEBUG] grad mean abs = 12.539852
[DEBUG] grad mean abs = 7.544260
[DEBUG] grad mean abs = 54.905579
[DEBUG] grad mean abs = 12.354552
[DEBUG] grad mean abs = 14.975005
[DEBUG] grad mean abs = 6.737308
[DEBUG] grad mean abs = 7.733046
[DEBUG] grad mean abs = 32.641870
[DEBUG] grad mean abs = 24.610350
[DEBUG] grad mean abs = 13.374803
[DEBUG] grad mean abs = 8.232310
[DEBUG] grad mean abs = 6.309531
[DEBUG] grad mean abs = 6.374425
[DEBUG] grad mean abs = 22.202474
[DEBUG] grad mean abs = 13.989982
[DEBUG] grad mean abs = 8.214391
[DEBUG] grad mean abs = 14.072491
[DEBUG] grad mean abs = 9.

KeyboardInterrupt: 

In [ ]:
from modeling_module.utils.checkpoint import load_model_dict
# Load
from modeling_module.models.model_builder import (
    build_patch_mixer_quantile,
    build_patchTST_base, build_patchTST_quantile, build_patch_mixer_base, build_titan_base, build_titan_lmm,
    build_titan_seq2seq,
)
device = 'cuda'

builders = {
    "patchmixer_base": build_patch_mixer_base,
    "patchmixer_quantile": build_patch_mixer_quantile,
    "titan_base": build_titan_base,
    "titan_lmm": build_titan_lmm,
    "titan_seq2seq": build_titan_seq2seq,
    "patchtst_base": build_patchTST_base,
    "patchtst_quantile": build_patchTST_quantile,
}
loaded = load_model_dict(save_dir, builders, device = device)


In [4]:
%load_ext autoreload
%autoreload 2

import importlib, modeling_module.utils.plot_utils as pu
import modeling_module.training.forecaster as fo
importlib.reload(pu)
importlib.reload(fo)

def my_exo_cb(start_idx: int, Hm: int, device="cuda" if torch.cuda.is_available() else "cpu"):
    # exo_dim = 2 (sin, cos)
    return fo.make_calendar_exo(start_idx, Hm, period=12, device=device)

pu.plot_120m(
    models=loaded,           # {"PatchMixer": pm_model, "Titan": ti_model, ...}
    loader=val_loader,       # (xb, yb[, part_ids])
    device="cuda" if torch.cuda.is_available() else "cpu",
    mode="val",              # ← 검증 모드
    max_plots=5,
    out_dir=None,
    show=True,
    future_exo_cb=my_exo_cb
)

NameError: name 'loaded' is not defined